In [15]:
!pip install tensorflow

In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [17]:
import shutil
from sklearn.model_selection import train_test_split

base_dir = 'jaundiced_normal'
jaundiced_dir = os.path.join(base_dir, 'Jaundiced Eyes')
normal_dir = os.path.join(base_dir, 'Normal Eyes')
working_dir = 'working/'

for split in ['train', 'val', 'test']:
    for category in ['Jaundiced Eyes', 'Normal Eyes']:
        os.makedirs(os.path.join(working_dir, split, category), exist_ok=True)

def split_and_copy(source_dir, category, split_ratio=(0.7, 0.15, 0.15)):
    images = os.listdir(source_dir)
    train_imgs, test_imgs = train_test_split(images, test_size=split_ratio[2], random_state=42)
    train_imgs, val_imgs = train_test_split(train_imgs, test_size=split_ratio[1]/(split_ratio[0] + split_ratio[1]), random_state=42)

    for img in train_imgs:
        shutil.copy(os.path.join(source_dir, img), os.path.join(working_dir, 'train', category, img))

    for img in val_imgs:
        shutil.copy(os.path.join(source_dir, img), os.path.join(working_dir, 'val', category, img))

    for img in test_imgs:
        shutil.copy(os.path.join(source_dir, img), os.path.join(working_dir, 'test', category, img))

split_and_copy(jaundiced_dir, 'Jaundiced Eyes')
split_and_copy(normal_dir, 'Normal Eyes')

In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_dir = os.path.join(working_dir, 'train')
val_dir = os.path.join(working_dir, 'val')
test_dir = os.path.join(working_dir, 'test')

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='binary')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode='binary')


Found 323 images belonging to 2 classes.
Found 70 images belonging to 2 classes.


In [19]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(train_generator, validation_data = val_generator, epochs = 10)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.5987 - loss: 1.8202 - val_accuracy: 0.6714 - val_loss: 0.5339
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 622ms/step - accuracy: 0.7859 - loss: 0.4527 - val_accuracy: 0.7429 - val_loss: 0.4602
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 631ms/step - accuracy: 0.8023 - loss: 0.4196 - val_accuracy: 0.8286 - val_loss: 0.3992
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 645ms/step - accuracy: 0.8157 - loss: 0.3799 - val_accuracy: 0.8286 - val_loss: 0.3641
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 626ms/step - accuracy: 0.8423 - loss: 0.3479 - val_accuracy: 0.7286 - val_loss: 0.5262
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 617ms/step - accuracy: 0.8145 - loss: 0.3736 - val_accuracy: 0.7714 - val_loss: 0.4866
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 645ms/step - accuracy: 0.8328 - loss: 0.3603 - val_accuracy: 0.7857 - val_loss: 0.3889
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 640ms/step - accuracy: 0.8905 - loss: 0.2976 - val_accuracy: 0.80

In [21]:
test_generator = val_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='binary')
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc:.4f}')

Found 71 images belonging to 2 classes.
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 778ms/step - accuracy: 0.8632 - loss: 0.3291
Test Accuracy: 0.8592


In [22]:
for layer in base_model.layers[:-5]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
model.fit(train_generator, validation_data=val_generator, epochs=10)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.8035 - loss: 0.4460 - val_accuracy: 0.8571 - val_loss: 0.3381
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 616ms/step - accuracy: 0.9505 - loss: 0.1382 - val_accuracy: 0.9000 - val_loss: 0.3654
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 603ms/step - accuracy: 0.9943 - loss: 0.0494 - val_accuracy: 0.8429 - val_loss: 0.6562
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 624ms/step - accuracy: 0.9851 - loss: 0.0637 - val_accuracy: 0.8714 - val_loss: 0.3553
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 584ms/step - accuracy: 0.9774 - loss: 0.0607 - val_accuracy: 0.9000 - val_loss: 0.2478
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 610ms/step - accuracy: 0.9920 - loss: 0.0357 - val_accuracy: 0.9143 - val_loss: 0.2422
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 617ms/step - accuracy: 0.9989 - loss: 0.0102 - val_accuracy: 0.9143 - val_loss: 0.2894
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 598ms/step - accuracy: 0.9973 - loss: 0.0132 - val_accuracy: 0.91

In [24]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc:.4f}')

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 820ms/step - accuracy: 0.9257 - loss: 0.3407
Test Accuracy: 0.9296


In [25]:
model.save('jaundice.keras')